### Задачи 1 и 2
Контрольная работа состоит из пяти вопросов. На каждый вопрос приведено четыре варианта ответа, один из которых правильный. Случайная величина X задаёт число правильных ответов при простом угадывании. Найдите математическое ожидание данной случайной величины. Найдите дисперсию случайной величины X.

In [36]:
#Функции для использования
import math as m
import pandas as pd
import numpy as np

# Сочетания
def combinations(k, n):
    c = m.factorial(n) // (m.factorial(k)*m.factorial(n - k))
    return c

# Формула Бернулли
def binomial(k, n, p):
    p_n = combinations(k, n)*pow(p, k)*pow((1-p),(n-k))
    return p_n

# Пуассоновское распределение
def pois(k, lambda_):
    p_k = pow(lambda_, k)*m.exp(-lambda_)/m.factorial(k)
    return p_k

In [37]:
k = []
for i in range(6):
    k.append(binomial(i, 5, 0.25))
df = pd.DataFrame(k, columns =['Вероятность'])
df

,Вероятность
0,0.237305
1,0.395508
2,0.263672
3,0.087891
4,0.014648
5,0.000977


In [38]:
print(f'M(X)={5*0.25}, D(X)={5*0.25*(1-0.25)}')

M(X)=1.25, D(X)=0.9375


### Задача 3
Пользователь получает в среднем 10 писем со спамом на свой почтовый ящик за сутки. Найти число N, такое, что с вероятностью 0.95 пользователь получит не более N писем со спамом за текущий день.

In [39]:
l = 10
k = 0
sum_p = 0
while sum_p < 0.95:
    sum_p += pois(k, l)
    k += 1

N = k - 1
print(N)

15


In [40]:
# проверка
sum(pois(i, l) for i in range(16)) 

0.9512595966960214

Посчитаем квантиль порядка 0,95 с помощью готовой функции.

In [41]:
from scipy import stats

In [42]:
stats.poisson.ppf(0.95, 10)

15.0

### Задача 4
Производятся выстрелы по мишени. Вероятность попадания в мишень при одном выстреле равна 0.01. Сколько выстрелов нужно сделать чтобы быть уверенным с вероятностью 0.9, что хотя бы 6 раз будет совершено попадание?

Подсказка: 1) "Вероятность попасть k раз при n выстрелах" - на какое распределение это похоже? 2) А если нам нужна вероятность P(X >= k), а не P(X = k)? 3) Здесь предстоит немножко покодить.

Вероятность попасть в мишень $K$ раз за $N$ выстрелов определяется формулой $$\displaystyle\sum_{k = K}^{N} C_N^k \cdot p^k \cdot (1-p)^{N-k}$$

Для нашей задачи $K=6$, $p=0.01$. Для ответа на вопрос, достаточно определить минимальное $N$, при котором $$\displaystyle\sum_{k = 0}^{K-1} C_N^k \cdot p^k \cdot (1-p)^{N-k} \leq 0.1$$

Определим функцию sum_comb(K, N, p), которая считает эту сумму и методом перебора найдём $N$.

In [58]:
"""число сочетаний из n по k"""
comb = lambda n, k: 0 if k > n else m.factorial(n) // (m.factorial(k)*m.factorial(n - k))
"""сумма вероятностеей попасть в мишень от 0 до K не включительно при N выстрелах с вероятностью попасть при одном выстреле p"""
def sum_comb(K: int, N: int, p: float) -> float:
    return sum(comb(N, k) * (p**k) * ((1-p)**(N-k)) for k in range(K))

In [59]:
N = 0
while True:
    if (sum_comb(6, N, 0.01) <= 0.1): break
    N += 1
print(N)

926
